In [0]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [0]:
import pandas as pd
from sklearn.utils import resample

df = pd.read_csv('original.csv',header=None)

In [58]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0


In [59]:
df[11].value_counts()

0    11847
1     2237
Name: 11, dtype: int64

In [0]:
zero = df[df[11] == 0]
one = df[df[11] == 1]

In [61]:
zero.shape

(11847, 12)

In [62]:
one.shape

(2237, 12)

In [0]:
new_zero = resample(zero, replace=False, n_samples=2237,random_state=42)

In [64]:
new_zero.shape

(2237, 12)

In [0]:
df1 = pd.concat([new_zero,one])

In [66]:
df1[11].value_counts()
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11
9786,30380,1188.0,1188,5.33,5.33,0,8.91,0.00,0.00,0,0,0
1204,20343,1188.0,1188,5.33,5.33,0,8.91,0.00,0.00,0,0,0
4846,4194,1620.0,1620,7.47,7.47,0,8.91,0.02,32.40,0,296,0
11647,3434,1188.0,1188,5.33,5.33,0,8.91,0.14,166.32,0,2,0
3567,10733,1620.0,1620,5.87,5.87,0,8.91,0.00,0.00,0,0,0


In [0]:
df1.to_csv('final_balanced.csv',index=False, header=False)

In [0]:
data = np.loadtxt('final_balanced.csv', delimiter=',')
unscaled_inp = data[:,1:-1]
target = data[:,-1]

In [0]:
scaled_inp = preprocessing.scale(unscaled_inp)

In [0]:
shuffled_indices = np.arange(scaled_inp.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inp = scaled_inp[shuffled_indices]
shuffled_targets = target[shuffled_indices]

In [71]:
sample_count = shuffled_inp.shape[0]

train_sample_count = int(0.8 * sample_count)
validation_sample_count = int(0.1 * sample_count)
test_sample_count = sample_count - train_sample_count - validation_sample_count

train_inp = shuffled_inp[:train_sample_count]
train_target = shuffled_targets[:train_sample_count]

validation_inp = shuffled_inp[train_sample_count:train_sample_count+validation_sample_count]
validation_target = shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_inp = shuffled_inp[train_sample_count+validation_sample_count:]
test_target = shuffled_targets[train_sample_count+validation_sample_count:]

print(np.sum(train_target)/train_sample_count)
print(np.sum(validation_target)/validation_sample_count)
print(np.sum(test_target)/test_sample_count)

0.5009779267951942
0.4809843400447427
0.5111607142857143


In [0]:
np.savez('Audibooks_data_train',inputs=train_inp, targets=train_target)
np.savez('Audibooks_data_valid',inputs=validation_inp, targets=validation_target)
np.savez('Audibooks_data_test',inputs=test_inp, targets=test_target)

In [0]:
npz = np.load('Audibooks_data_train.npz')
train_inp = npz['inputs'].astype(np.float)
train_target = npz['targets'].astype(np.float)

npz = np.load('Audibooks_data_valid.npz')
valid_inp = npz['inputs'].astype(np.float)
valid_target = npz['targets'].astype(np.float)

npz = np.load('Audibooks_data_test.npz')
test_inp = npz['inputs'].astype(np.float)
test_target = npz['targets'].astype(np.float)

In [79]:
input_size = 10     # 10 Predictors
hidden_size = 50
output_size = 2

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(32, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
])

adam = tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

model.compile(optimizer=adam,loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping()

batch_size = 100
max_epochs = 100
model.fit(train_inp,train_target,batch_size=batch_size,
          callbacks=[early_stopping],epochs=max_epochs, validation_data=(valid_inp,valid_target),verbose=2)

Epoch 1/100
36/36 - 0s - loss: 0.7190 - accuracy: 0.5823 - val_loss: 0.6970 - val_accuracy: 0.5861
Epoch 2/100
36/36 - 0s - loss: 0.6990 - accuracy: 0.5993 - val_loss: 0.6786 - val_accuracy: 0.6197
Epoch 3/100
36/36 - 0s - loss: 0.6807 - accuracy: 0.6194 - val_loss: 0.6614 - val_accuracy: 0.6465
Epoch 4/100
36/36 - 0s - loss: 0.6639 - accuracy: 0.6393 - val_loss: 0.6460 - val_accuracy: 0.6868
Epoch 5/100
36/36 - 0s - loss: 0.6485 - accuracy: 0.6583 - val_loss: 0.6316 - val_accuracy: 0.7092
Epoch 6/100
36/36 - 0s - loss: 0.6343 - accuracy: 0.6778 - val_loss: 0.6185 - val_accuracy: 0.7159
Epoch 7/100
36/36 - 0s - loss: 0.6212 - accuracy: 0.6963 - val_loss: 0.6066 - val_accuracy: 0.7315
Epoch 8/100
36/36 - 0s - loss: 0.6094 - accuracy: 0.7117 - val_loss: 0.5951 - val_accuracy: 0.7383
Epoch 9/100
36/36 - 0s - loss: 0.5982 - accuracy: 0.7220 - val_loss: 0.5847 - val_accuracy: 0.7562
Epoch 10/100
36/36 - 0s - loss: 0.5879 - accuracy: 0.7332 - val_loss: 0.5752 - val_accuracy: 0.7673
Epoch 11/

In [80]:
test_loss, test_accuracy = model.evaluate(test_inp, test_target)

14/14 [==============================] - 0s 2ms/step - loss: 0.3763 - accuracy: 0.8192
